In [5]:
#import libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
import pandas as pd
import sklearn

In [6]:
%store -r cook_county_gdf
%store -r CC_nodeaths

In [31]:
columns_to_keep = ['STATEFP', 'COUNTYFP', 'TRACTCE', 'NAME', 'NAMELSAD', 'MTFCC',
                   'FUNCSTAT', 'ALAND', 'AWATER', 'latitude', 'longitude', 'geometry']

gdf_annual = cook_county_gdf.groupby(['GEOID', 'year'])[columns_to_keep].first()

#this does not sum deaths? 

KeyError: "None of ['GEOID', 'year'] are in the columns"

In [25]:
gdf_annual

STATEFP COUNTYFP TRACTCE  NAME           NAMELSAD  MTFCC  \
GEOID       year                                                            
17031010100 2014      17      031  010100   101   Census Tract 101  G5020   
            2015      17      031  010100   101   Census Tract 101  G5020   
            2016      17      031  010100   101   Census Tract 101  G5020   
            2017      17      031  010100   101   Census Tract 101  G5020   
            2019      17      031  010100   101   Census Tract 101  G5020   
...                  ...      ...     ...   ...                ...    ...   
17031980000 2021      17      031  980000  9800  Census Tract 9800  G5020   
            2022      17      031  980000  9800  Census Tract 9800  G5020   
            2023      17      031  980000  9800  Census Tract 9800  G5020   
17031980100 2018      17      031  980100  9801  Census Tract 9801  G5020   
            2022      17      031  980100  9801  Census Tract 9801  G5020   

                 FUNCSTAT       ALAND   AWATER   latitude  longitude  \
GEOID       year                                                       
17031010100 2014        S    379511.0      0.0  42.022129 -87.666329   
            2015        S    379511.0      0.0  42.019895 -87.672027   
            2016        S    379511.0      0.0  42.021442 -87.671426   
            2017        S    379511.0      0.0  42.022168 -87.668053   
            2019        S    379511.0      0.0  42.022326 -87.667395   
...                   ...         ...      ...        ...        ...   
17031980000 2021        S  19890200.0  92402.0  41.976125 -87.905086   
            2022        S  19890200.0  92402.0  41.976844 -87.906479   
            2023        S  19890200.0  92402.0  41.975732 -87.903989   
17031980100 2018        S   2981781.0      0.0  41.789524 -87.741373   
            2022        S   2981781.0      0.0  41.786723 -87.738049   

                                    geometry  
GEOID       year                              
17031010100 2014  POINT (-87.66632 42.02212)  
            2015  POINT (-87.67202 42.01989)  
            2016  POINT (-87.67142 42.02143)  
            2017  POINT (-87.66805 42.02216)  
            2019  POINT (-87.66739 42.02232)  
...                                      ...  
17031980000 2021  POINT (-87.90508 41.97612)  
            2022  POINT (-87.90647 41.97684)  
            2023  POINT (-87.90398 41.97572)  
17031980100 2018  POINT (-87.74137 41.78952)  
            2022  POINT (-87.73804 41.78672)  

[5764 rows x 12 columns]

In [26]:
unique_geoid_count = gdf_annual.index.get_level_values('GEOID').nunique()
print(unique_geoid_count) #missing 58 tracts

1274


### Adding rows for zero-death tracts & years

gdf_annnual does not include rows for census tracts or years that have zero deaths. the below code finds the missing census tracts and years and creates a row for it in the dataframe. 

In [28]:
CC_nodeaths.columns

Index(['STATEFP', 'COUNTYFP', 'TRACTCE', 'GEOID', 'NAME', 'NAMELSAD', 'MTFCC',
       'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON', 'geometry',
       'deaths'],
      dtype='object')

In [29]:
gdf_annual.columns

Index(['STATEFP', 'COUNTYFP', 'TRACTCE', 'NAME', 'NAMELSAD', 'MTFCC',
       'FUNCSTAT', 'ALAND', 'AWATER', 'latitude', 'longitude', 'geometry'],
      dtype='object')

In [30]:
##below chunk of code adds in any tracts that had 0 deaths over the course of 2014-2023
##loop through CC_nodeaths - for every row, we want to add 10 rows to gdf_annual

#extract 'GEOID' from CC_nodeaths, add it as a new index with 10 years as well to gdf_annual, duplicate values
##for 'STATEFP', 'COUNTYFP', 'TRACTCE', 'NAME', 'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 
##'INTPTLAT', 'INTPTLON', 'geometry'


new_rows = []

for _, row in CC_nodeaths.iterrows():
    tract_geoID = row['GEOID']

    for year in range(2014, 2024):
        if (tract_geoID, year) not in gdf_annual.index:
            new_rows.append([tract_geoID, year] + [None] * (len(gdf_annual.columns) - 2))

new_rows_df = pd.DataFrame(new_rows, columns=gdf_annual.columns)
gdf_annual = pd.concat([gdf_annual, new_rows_df]).sort_index()


TypeError: '<' not supported between instances of 'int' and 'tuple'

In [ ]:
gdf_annual

In [ ]:
unique_geoid_count = gdf_annual.index.get_level_values('GEOID').nunique()
print(unique_geoid_count) 

#yay - this matches the # of tracts in the shapefile from census. 

In [ ]:
##below chunk of code goes through all tracts and ensures it has a row for each year b/t 2014-2023, if not,
## we add the year and set the number of deaths equal to 0

years = range(2014, 2024)
updated_rows = []

for tract_geoID in gdf_annual['GEOID'].unique():
    existing_years = set(gdf_annual[gdf_annual['GEOID'] == tract_geoID]['year'])
    missing_years = set(years) - existing_years
    
    for year in missing_years:
        updated_rows.append({'GEOID': tract_geoID, 'year': year, 'deaths_sum': 0})

gdf_annual = pd.concat([gdf_annual, pd.DataFrame(updated_rows)], ignore_index=True)


In [ ]:
gdf_annual

In [ ]:
#df_annual = gdf_annual.set_index(['tract_geoID', 'year'])
#df_annual.shape

## Tables & Graphs

In [ ]:
sparsity_df = pd.DataFrame(columns=['year', 'Total Deaths', 'Sparsity Rate'])

years = range(2014, 2023)
for year in years:
    year_df = gdf_annual[gdf_annual['year'] == year]
    zero_deaths_count = year_df[year_df['deaths_sum'] == 0].shape[0]
    total_count = year_df.shape[0]
    sparsity_rate = (zero_deaths_count / total_count) * 100
    total_deaths = year_df['deaths_sum'].sum()
    sparsity_df = pd.concat([sparsity_df, pd.DataFrame({'year': [year], 'Total Deaths': [total_deaths], 'Sparsity Rate': [sparsity_rate]})], ignore_index=True)


raw_sparsity = sparsity_df['Sparsity Rate']
sparsity_df['Sparsity Rate'] = sparsity_df['Sparsity Rate'].round(3).astype(str) + '%'
print(sparsity_df)

In [ ]:
plt.figure(figsize=(9, 6))
df_no23 = gdf_annual[df_annual['year'] != 2023]
sns.boxplot(x='year', y='deaths_sum', data=df_no23, color='lightblue')

plt.xlabel('Year')
plt.ylabel('Number of Deaths')
plt.title('Distribution of Deaths Across Years')
plt.show()


In [ ]:
plt.plot(range(2014, 2023), raw_sparsity, color='mediumslateblue')

font = {'family': 'Times New Roman',
        'weight': 'normal',
        'size': 12}
plt.rc('font', **font)


plt.xlabel('Year')
plt.ylabel('Sparsity Rate')
plt.title('Sparsity Rates Over the Years')
plt.grid(False)

plt.show()